
#### Run the cell below to install the required packages for Copilot


In [41]:
!pip install semantic-link --q

import sempy.fabric as fabric
import base64
import time
import uuid
import json

StatementMeta(, a40c28b3-b5ab-445c-a790-10036d0850e9, 43, Finished, Available, Finished)

In [42]:
client = fabric.FabricRestClient()
workspaceId = fabric.get_workspace_id()
randomId=uuid.uuid4()
EventhouseName = f"{'SampleEventhouse'}_{randomId}"
EventstreamName = f"{'SampleEventstream'}_{randomId}"
DataActivatorName = f"{'SampleDataActivator'}_{randomId}"
DBName=f"{'TaxiDB'}_{randomId}"
DBCache="P30D"
DBStorage="P365D"

StatementMeta(, a40c28b3-b5ab-445c-a790-10036d0850e9, 44, Finished, Available, Finished)

In [43]:
url = f"v1/workspaces/{workspaceId}/reflexes"

payload = {
"displayName": f"{DataActivatorName}"
}

response=client.post(url,json=payload)

DataActivatorId=response.json()['id']

StatementMeta(, a40c28b3-b5ab-445c-a790-10036d0850e9, 45, Finished, Available, Finished)

In [44]:
url = f"v1/workspaces/{workspaceId}/eventhouses"

payload = {
    "displayName": f"{EventhouseName}"
}

response=client.post(url,json=payload)

EventhouseId=response.json()['id']

StatementMeta(, a40c28b3-b5ab-445c-a790-10036d0850e9, 46, Finished, Available, Finished)

In [ ]:
url = f"v1/workspaces/{workspaceId}/kqlDatabases"

response=client.get(url)

print(response.json())

for item in response.json()['value']:
    if item['displayName'] == f"{EventhouseName}":
        url = f"v1/workspaces/{workspaceId}/kqlDatabases/{item['id']}"
        client.delete(url)

In [46]:
url = f"v1/workspaces/{workspaceId}/kqlDatabases"

dbproperties={
  "databaseType": "ReadWrite",
  "parentEventhouseItemId": f"{EventhouseId}", 
  "oneLakeCachingPeriod": f"{DBCache}", 
  "oneLakeStandardStoragePeriod": f"{DBStorage}" 
}

dbproperties = json.dumps(dbproperties)


dbschema=""".create-merge table TaxiRaw (VendorID:string, tpep_pickup_datetime:datetime, tpep_dropoff_datetime:datetime, passenger_count:real, trip_distance:real, RatecodeID:real, store_and_fwd_flag:string, PULocationID:string, DOLocationID:string, payment_type:long, fare_amount:real, extra:real, mta_tax:real, tip_amount:real, tolls_amount:real, improvement_surcharge:real, total_amount:real, congestion_surcharge:real, airport_fee:real) with (folder = "", docstring = "")  
.alter table TaxiRaw policy streamingingestion disable 
.alter table TaxiRaw policy retention @'{"SoftDeletePeriod":"1.00:00:00","Recoverability":"Enabled"}'
.alter table TaxiRaw policy caching hot = 1d
.create-or-alter table TaxiRaw ingestion json mapping 'TaxiRaw_mapping' ```[{"column":"VendorID","path":"$['VendorID']","datatype":""},{"column":"tpep_pickup_datetime","path":"$['tpep_pickup_datetime']","datatype":""},{"column":"tpep_dropoff_datetime","path":"$['tpep_dropoff_datetime']","datatype":""},{"column":"passenger_count","path":"$['passenger_count']","datatype":""},{"column":"trip_distance","path":"$['trip_distance']","datatype":""},{"column":"RatecodeID","path":"$['RatecodeID']","datatype":""},{"column":"store_and_fwd_flag","path":"$['store_and_fwd_flag']","datatype":""},{"column":"PULocationID","path":"$['PULocationID']","datatype":""},{"column":"DOLocationID","path":"$['DOLocationID']","datatype":""},{"column":"payment_type","path":"$['payment_type']","datatype":""},{"column":"fare_amount","path":"$['fare_amount']","datatype":""},{"column":"extra","path":"$['extra']","datatype":""},{"column":"mta_tax","path":"$['mta_tax']","datatype":""},{"column":"tip_amount","path":"$['tip_amount']","datatype":""},{"column":"tolls_amount","path":"$['tolls_amount']","datatype":""},{"column":"improvement_surcharge","path":"$['improvement_surcharge']","datatype":""},{"column":"total_amount","path":"$['total_amount']","datatype":""},{"column":"congestion_surcharge","path":"$['congestion_surcharge']","datatype":""},{"column":"airport_fee","path":"$['airport_fee']","datatype":""}]```
.create-merge table ZoneLookup (LocationID:string,	Borough:string,	Zone:string,	service_zone:string)
.create-or-alter function with(skipvalidation=true) TaxiUpdate() {TaxiRaw | lookup (ZoneLookup | project DOLocationID=LocationID, DOBourough=Borough, DOZone=Zone, DOService_Zone=service_zone) on DOLocationID | lookup (ZoneLookup | project PULocationID=LocationID, PUBourough=Borough, PUZone=Zone, PUService_Zone=service_zone) on PULocationID }
.create-merge table TaxiRecords (VendorID:string, tpep_pickup_datetime:datetime, tpep_dropoff_datetime:datetime, passenger_count:real, trip_distance:real, RatecodeID:real, store_and_fwd_flag:string, PULocationID:string, DOLocationID:string, payment_type:long, fare_amount:real, extra:real, mta_tax:real, tip_amount:real, tolls_amount:real, improvement_surcharge:real, total_amount:real, congestion_surcharge:real, airport_fee:real, DOBourough:string, DOZone:string, DOService_Zone:string, PUBourough:string, PUZone:string, PUService_Zone:string) 
.alter table TaxiRecords policy update ```[{"IsEnabled": true, "Source": "TaxiRaw", "Query": "TaxiUpdate()", "IsTransactional": true, "PropagateIngestionProperties": false }]```
.alter table TaxiRecords policy retention @'{"SoftDeletePeriod":"10.00:00:00","Recoverability":"Enabled"}'
.alter table TaxiRecords policy caching hot = 3d
.create-or-alter materialized-view TaxiRecordsDedup on table TaxiRecords {TaxiRecords | summarize take_any(*) by VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, PULocationID, DOLocationID, trip_distance }
.alter materialized-view  TaxiRecordsDedup policy retention @'{"SoftDeletePeriod":"30.00:00:00","Recoverability":"Enabled"}'
.alter materialized-view TaxiRecordsDedup policy caching hot = 14d
.create-or-alter materialized-view TaxiRecordsHourly on materialized-view TaxiRecordsDedup {TaxiRecordsDedup | summarize Avg_Passenger_Count=avg(passenger_count), Avg_Trip_Distance=avg(trip_distance), Avg_Fare_Amount=avg(fare_amount), Avg_Extra=avg(extra), Avg_MTA_Tax=avg(mta_tax), Avg_Tip_Amount=avg(tip_amount), Avg_Tolls_Amount=avg(tolls_amount), Avg_Improvement_Surcharge=avg(improvement_surcharge), Avg_Total_Amount=avg(total_amount), Avg_Congestion_Surcharge=avg(congestion_surcharge), Avg_Airport_Fee=avg(airport_fee) by bin(tpep_pickup_datetime,1h), bin(tpep_dropoff_datetime,1h), store_and_fwd_flag, PULocationID, DOLocationID, payment_type, DOBourough, DOZone, DOService_Zone, PUBourough, PUZone, PUService_Zone}
.alter materialized-view  TaxiRecordsHourly policy retention @'{"SoftDeletePeriod":"365.00:00:00","Recoverability":"Enabled"}'
.alter materialized-view TaxiRecordsHourly policy caching hot = 60d
"""

dbproperties_string = dbproperties.encode('utf-8')
dbproperties_bytes = base64.b64encode(dbproperties_string)
dbproperties_string = dbproperties_bytes.decode('utf-8')

dbschema_string = dbschema.encode('utf-8')
dbschema_bytes = base64.b64encode(dbschema_string)
dbschema_string = dbschema_bytes.decode('utf-8')


StatementMeta(, a40c28b3-b5ab-445c-a790-10036d0850e9, 48, Finished, Available, Finished)

In [47]:
url = f"v1/workspaces/{workspaceId}/kqlDatabases"

payload = {
    "displayName": f"{DBName}",
    "definition": {
      "parts": [
        {
          "path": "DatabaseProperties.json",
          "payload": f"{dbproperties_string}",
          "payloadType": "InlineBase64"
        },
        {
          "path": "DatabaseSchema.kql",
          "payload": f"{dbschema_string}",
          "payloadType": "InlineBase64"
        }
      ]
  }
}

response=client.post(url,json=payload)

StatementMeta(, a40c28b3-b5ab-445c-a790-10036d0850e9, 49, Finished, Available, Finished)

In [ ]:
print(f"Create request status code {response.status_code}")
print(response.headers['Location'])
async_result_polling_url = response.headers['Location']

while True:
    async_response = client.get(async_result_polling_url)
    async_status = async_response.json().get('status').lower()
    print("Long running operation status " + async_status)
    if async_status != 'running':
        break
   
    time.sleep(3)

print("Long running operation reached terminal state '" + async_status +"'")

if async_status == 'succeeded':
    print("The operation completed successfully.")
    final_result_url= async_response.headers['Location']
    final_result = client.get(final_result_url)
    print(f"Final result: {final_result.json()}")
elif async_status == 'failed':
    print("The operation failed.")
else:
    print("The operation is in an unexpected state:", status)

dbId=final_result.json()['id']


In [49]:
import requests
url = f"v1/workspaces/{workspaceId}/eventhouses/{EventhouseId}"

response=client.get(url)

queryURI=response.json()['properties']['queryServiceUri']
HostName=queryURI.replace("https://", "")

url = f"{queryURI}/v1/rest/mgmt"
token_string = mssparkutils.credentials.getToken(f"{queryURI}")
header = {'Content-Type':'application/json','Authorization': f'Bearer {token_string}'}

cmd=""".ingest inline into table ZoneLookup <|
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone
6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone
7,Queens,Astoria,Boro Zone
8,Queens,Astoria Park,Boro Zone
9,Queens,Auburndale,Boro Zone
10,Queens,Baisley Park,Boro Zone
11,Brooklyn,Bath Beach,Boro Zone
12,Manhattan,Battery Park,Yellow Zone
13,Manhattan,Battery Park City,Yellow Zone
14,Brooklyn,Bay Ridge,Boro Zone
15,Queens,Bay Terrace/Fort Totten,Boro Zone
16,Queens,Bayside,Boro Zone
17,Brooklyn,Bedford,Boro Zone
18,Bronx,Bedford Park,Boro Zone
19,Queens,Bellerose,Boro Zone
20,Bronx,Belmont,Boro Zone
21,Brooklyn,Bensonhurst East,Boro Zone
22,Brooklyn,Bensonhurst West,Boro Zone
23,Staten Island,Bloomfield/Emerson Hill,Boro Zone
24,Manhattan,Bloomingdale,Yellow Zone
25,Brooklyn,Boerum Hill,Boro Zone
26,Brooklyn,Borough Park,Boro Zone
27,Queens,Breezy Point/Fort Tilden/Riis Beach,Boro Zone
28,Queens,Briarwood/Jamaica Hills,Boro Zone
29,Brooklyn,Brighton Beach,Boro Zone
30,Queens,Broad Channel,Boro Zone
31,Bronx,Bronx Park,Boro Zone
32,Bronx,Bronxdale,Boro Zone
33,Brooklyn,Brooklyn Heights,Boro Zone
34,Brooklyn,Brooklyn Navy Yard,Boro Zone
35,Brooklyn,Brownsville,Boro Zone
36,Brooklyn,Bushwick North,Boro Zone
37,Brooklyn,Bushwick South,Boro Zone
38,Queens,Cambria Heights,Boro Zone
39,Brooklyn,Canarsie,Boro Zone
40,Brooklyn,Carroll Gardens,Boro Zone
41,Manhattan,Central Harlem,Boro Zone
42,Manhattan,Central Harlem North,Boro Zone
43,Manhattan,Central Park,Yellow Zone
44,Staten Island,Charleston/Tottenville,Boro Zone
45,Manhattan,Chinatown,Yellow Zone
46,Bronx,City Island,Boro Zone
47,Bronx,Claremont/Bathgate,Boro Zone
48,Manhattan,Clinton East,Yellow Zone
49,Brooklyn,Clinton Hill,Boro Zone
50,Manhattan,Clinton West,Yellow Zone
51,Bronx,Co-Op City,Boro Zone
52,Brooklyn,Cobble Hill,Boro Zone
53,Queens,College Point,Boro Zone
54,Brooklyn,Columbia Street,Boro Zone
55,Brooklyn,Coney Island,Boro Zone
56,Queens,Corona,Boro Zone
57,Queens,Corona,Boro Zone
58,Bronx,Country Club,Boro Zone
59,Bronx,Crotona Park,Boro Zone
60,Bronx,Crotona Park East,Boro Zone
61,Brooklyn,Crown Heights North,Boro Zone
62,Brooklyn,Crown Heights South,Boro Zone
63,Brooklyn,Cypress Hills,Boro Zone
64,Queens,Douglaston,Boro Zone
65,Brooklyn,Downtown Brooklyn/MetroTech,Boro Zone
66,Brooklyn,DUMBO/Vinegar Hill,Boro Zone
67,Brooklyn,Dyker Heights,Boro Zone
68,Manhattan,East Chelsea,Yellow Zone
69,Bronx,East Concourse/Concourse Village,Boro Zone
70,Queens,East Elmhurst,Boro Zone
71,Brooklyn,East Flatbush/Farragut,Boro Zone
72,Brooklyn,East Flatbush/Remsen Village,Boro Zone
73,Queens,East Flushing,Boro Zone
74,Manhattan,East Harlem North,Boro Zone
75,Manhattan,East Harlem South,Boro Zone
76,Brooklyn,East New York,Boro Zone
77,Brooklyn,East New York/Pennsylvania Avenue,Boro Zone
78,Bronx,East Tremont,Boro Zone
79,Manhattan,East Village,Yellow Zone
80,Brooklyn,East Williamsburg,Boro Zone
81,Bronx,Eastchester,Boro Zone
82,Queens,Elmhurst,Boro Zone
83,Queens,Elmhurst/Maspeth,Boro Zone
84,Staten Island,Eltingville/Annadale/Princes Bay,Boro Zone
85,Brooklyn,Erasmus,Boro Zone
86,Queens,Far Rockaway,Boro Zone
87,Manhattan,Financial District North,Yellow Zone
88,Manhattan,Financial District South,Yellow Zone
89,Brooklyn,Flatbush/Ditmas Park,Boro Zone
90,Manhattan,Flatiron,Yellow Zone
91,Brooklyn,Flatlands,Boro Zone
92,Queens,Flushing,Boro Zone
93,Queens,Flushing Meadows-Corona Park,Boro Zone
94,Bronx,Fordham South,Boro Zone
95,Queens,Forest Hills,Boro Zone
96,Queens,Forest Park/Highland Park,Boro Zone
97,Brooklyn,Fort Greene,Boro Zone
98,Queens,Fresh Meadows,Boro Zone
99,Staten Island,Freshkills Park,Boro Zone
100,Manhattan,Garment District,Yellow Zone
101,Queens,Glen Oaks,Boro Zone
102,Queens,Glendale,Boro Zone
103,Manhattan,Governors Island/Ellis Island/Liberty Island,Yellow Zone
104,Manhattan,Governors Island/Ellis Island/Liberty Island,Yellow Zone
105,Manhattan,Governors Island/Ellis Island/Liberty Island,Yellow Zone
106,Brooklyn,Gowanus,Boro Zone
107,Manhattan,Gramercy,Yellow Zone
108,Brooklyn,Gravesend,Boro Zone
109,Staten Island,Great Kills,Boro Zone
110,Staten Island,Great Kills Park,Boro Zone
111,Brooklyn,Green-Wood Cemetery,Boro Zone
112,Brooklyn,Greenpoint,Boro Zone
113,Manhattan,Greenwich Village North,Yellow Zone
114,Manhattan,Greenwich Village South,Yellow Zone
115,Staten Island,Grymes Hill/Clifton,Boro Zone
116,Manhattan,Hamilton Heights,Boro Zone
117,Queens,Hammels/Arverne,Boro Zone
118,Staten Island,Heartland Village/Todt Hill,Boro Zone
119,Bronx,Highbridge,Boro Zone
120,Manhattan,Highbridge Park,Boro Zone
121,Queens,Hillcrest/Pomonok,Boro Zone
122,Queens,Hollis,Boro Zone
123,Brooklyn,Homecrest,Boro Zone
124,Queens,Howard Beach,Boro Zone
125,Manhattan,Hudson Sq,Yellow Zone
126,Bronx,Hunts Point,Boro Zone
127,Manhattan,Inwood,Boro Zone
128,Manhattan,Inwood Hill Park,Boro Zone
129,Queens,Jackson Heights,Boro Zone
130,Queens,Jamaica,Boro Zone
131,Queens,Jamaica Estates,Boro Zone
132,Queens,JFK Airport,Airports
133,Brooklyn,Kensington,Boro Zone
134,Queens,Kew Gardens,Boro Zone
135,Queens,Kew Gardens Hills,Boro Zone
136,Bronx,Kingsbridge Heights,Boro Zone
137,Manhattan,Kips Bay,Yellow Zone
138,Queens,LaGuardia Airport,Airports
139,Queens,Laurelton,Boro Zone
140,Manhattan,Lenox Hill East,Yellow Zone
141,Manhattan,Lenox Hill West,Yellow Zone
142,Manhattan,Lincoln Square East,Yellow Zone
143,Manhattan,Lincoln Square West,Yellow Zone
144,Manhattan,Little Italy/NoLiTa,Yellow Zone
145,Queens,Long Island City/Hunters Point,Boro Zone
146,Queens,Long Island City/Queens Plaza,Boro Zone
147,Bronx,Longwood,Boro Zone
148,Manhattan,Lower East Side,Yellow Zone
149,Brooklyn,Madison,Boro Zone
150,Brooklyn,Manhattan Beach,Boro Zone
151,Manhattan,Manhattan Valley,Yellow Zone
152,Manhattan,Manhattanville,Boro Zone
153,Manhattan,Marble Hill,Boro Zone
154,Brooklyn,Marine Park/Floyd Bennett Field,Boro Zone
155,Brooklyn,Marine Park/Mill Basin,Boro Zone
156,Staten Island,Mariners Harbor,Boro Zone
157,Queens,Maspeth,Boro Zone
158,Manhattan,Meatpacking/West Village West,Yellow Zone
159,Bronx,Melrose South,Boro Zone
160,Queens,Middle Village,Boro Zone
161,Manhattan,Midtown Center,Yellow Zone
162,Manhattan,Midtown East,Yellow Zone
163,Manhattan,Midtown North,Yellow Zone
164,Manhattan,Midtown South,Yellow Zone
165,Brooklyn,Midwood,Boro Zone
166,Manhattan,Morningside Heights,Boro Zone
167,Bronx,Morrisania/Melrose,Boro Zone
168,Bronx,Mott Haven/Port Morris,Boro Zone
169,Bronx,Mount Hope,Boro Zone
170,Manhattan,Murray Hill,Yellow Zone
171,Queens,Murray Hill-Queens,Boro Zone
172,Staten Island,New Dorp/Midland Beach,Boro Zone
173,Queens,North Corona,Boro Zone
174,Bronx,Norwood,Boro Zone
175,Queens,Oakland Gardens,Boro Zone
176,Staten Island,Oakwood,Boro Zone
177,Brooklyn,Ocean Hill,Boro Zone
178,Brooklyn,Ocean Parkway South,Boro Zone
179,Queens,Old Astoria,Boro Zone
180,Queens,Ozone Park,Boro Zone
181,Brooklyn,Park Slope,Boro Zone
182,Bronx,Parkchester,Boro Zone
183,Bronx,Pelham Bay,Boro Zone
184,Bronx,Pelham Bay Park,Boro Zone
185,Bronx,Pelham Parkway,Boro Zone
186,Manhattan,Penn Station/Madison Sq West,Yellow Zone
187,Staten Island,Port Richmond,Boro Zone
188,Brooklyn,Prospect-Lefferts Gardens,Boro Zone
189,Brooklyn,Prospect Heights,Boro Zone
190,Brooklyn,Prospect Park,Boro Zone
191,Queens,Queens Village,Boro Zone
192,Queens,Queensboro Hill,Boro Zone
193,Queens,Queensbridge/Ravenswood,Boro Zone
194,Manhattan,Randalls Island,Yellow Zone
195,Brooklyn,Red Hook,Boro Zone
196,Queens,Rego Park,Boro Zone
197,Queens,Richmond Hill,Boro Zone
198,Queens,Ridgewood,Boro Zone
199,Bronx,Rikers Island,Boro Zone
200,Bronx,Riverdale/North Riverdale/Fieldston,Boro Zone
201,Queens,Rockaway Park,Boro Zone
202,Manhattan,Roosevelt Island,Boro Zone
203,Queens,Rosedale,Boro Zone
204,Staten Island,Rossville/Woodrow,Boro Zone
205,Queens,Saint Albans,Boro Zone
206,Staten Island,Saint George/New Brighton,Boro Zone
207,Queens,Saint Michaels Cemetery/Woodside,Boro Zone
208,Bronx,Schuylerville/Edgewater Park,Boro Zone
209,Manhattan,Seaport,Yellow Zone
210,Brooklyn,Sheepshead Bay,Boro Zone
211,Manhattan,SoHo,Yellow Zone
212,Bronx,Soundview/Bruckner,Boro Zone
213,Bronx,Soundview/Castle Hill,Boro Zone
214,Staten Island,South Beach/Dongan Hills,Boro Zone
215,Queens,South Jamaica,Boro Zone
216,Queens,South Ozone Park,Boro Zone
217,Brooklyn,South Williamsburg,Boro Zone
218,Queens,Springfield Gardens North,Boro Zone
219,Queens,Springfield Gardens South,Boro Zone
220,Bronx,Spuyten Duyvil/Kingsbridge,Boro Zone
221,Staten Island,Stapleton,Boro Zone
222,Brooklyn,Starrett City,Boro Zone
223,Queens,Steinway,Boro Zone
224,Manhattan,Stuy Town/Peter Cooper Village,Yellow Zone
225,Brooklyn,Stuyvesant Heights,Boro Zone
226,Queens,Sunnyside,Boro Zone
227,Brooklyn,Sunset Park East,Boro Zone
228,Brooklyn,Sunset Park West,Boro Zone
229,Manhattan,Sutton Place/Turtle Bay North,Yellow Zone
230,Manhattan,Times Sq/Theatre District,Yellow Zone
231,Manhattan,TriBeCa/Civic Center,Yellow Zone
232,Manhattan,Two Bridges/Seward Park,Yellow Zone
233,Manhattan,UN/Turtle Bay South,Yellow Zone
234,Manhattan,Union Sq,Yellow Zone
235,Bronx,University Heights/Morris Heights,Boro Zone
236,Manhattan,Upper East Side North,Yellow Zone
237,Manhattan,Upper East Side South,Yellow Zone
238,Manhattan,Upper West Side North,Yellow Zone
239,Manhattan,Upper West Side South,Yellow Zone
240,Bronx,Van Cortlandt Park,Boro Zone
241,Bronx,Van Cortlandt Village,Boro Zone
242,Bronx,Van Nest/Morris Park,Boro Zone
243,Manhattan,Washington Heights North,Boro Zone
244,Manhattan,Washington Heights South,Boro Zone
245,Staten Island,West Brighton,Boro Zone
246,Manhattan,West Chelsea/Hudson Yards,Yellow Zone
247,Bronx,West Concourse,Boro Zone
248,Bronx,West Farms/Bronx River,Boro Zone
249,Manhattan,West Village,Yellow Zone
250,Bronx,Westchester Village/Unionport,Boro Zone
251,Staten Island,Westerleigh,Boro Zone
252,Queens,Whitestone,Boro Zone
253,Queens,Willets Point,Boro Zone
254,Bronx,Williamsbridge/Olinville,Boro Zone
255,Brooklyn,Williamsburg (North Side),Boro Zone
256,Brooklyn,Williamsburg (South Side),Boro Zone
257,Brooklyn,Windsor Terrace,Boro Zone
258,Queens,Woodhaven,Boro Zone
259,Bronx,Woodlawn/Wakefield,Boro Zone
260,Queens,Woodside,Boro Zone
261,Manhattan,World Trade Center,Yellow Zone
262,Manhattan,Yorkville East,Yellow Zone
263,Manhattan,Yorkville West,Yellow Zone
264,Unknown,N/A,N/A
265,N/A,Outside of NYC,N/A
"""

payload = {
    "csl": f"{cmd}",
    "db": f"{DBName}"
}

response=requests.post(url,json=payload, headers=header)



StatementMeta(, a40c28b3-b5ab-445c-a790-10036d0850e9, 51, Finished, Available, Finished)

In [50]:
eventStreamPlatform={
  "$schema": "https://developer.microsoft.com/json-schemas/fabric/gitIntegration/platformProperties/2.0.0/schema.json",
  "metadata": {
    "type": "Eventstream",
    "displayName": f"{EventstreamName}"
  },
  "config": {
    "version": "2.0",
    "logicalId": "00000000-0000-0000-0000-000000000000"
  }    
}

eventStreamPlatform = json.dumps(eventStreamPlatform)

eventStreamPlatform_string = eventStreamPlatform.encode('utf-8')
eventStreamPlatform_bytes = base64.b64encode(eventStreamPlatform_string)
eventStreamPlatform_string = eventStreamPlatform_bytes.decode('utf-8')

eventStreamProperties={
  "sources": [
    {
      "name": "Taxi",
      "type": "SampleData",
      "properties": {
        "type": "YellowTaxi"
      }
    }
  ],
  "destinations": [
    {
      "name": "Eventhouse",
      "type": "Eventhouse",
      "properties": {
        "dataIngestionMode": "ProcessedIngestion",
        "workspaceId": f"{workspaceId}",
        "itemId": f"{dbId}",
        "databaseName": f"{DBName}",
        "tableName": "TaxiRaw",
        "inputSerialization": {
          "type": "Json",
          "properties": {
            "encoding": "UTF8"
          }
        }
      },
      "inputNodes": [
        {
          "name": "ManageFields"
        }
      ],
      "inputSchemas": [
        {
          "name": "ManageFields",
          "schema": {
            "columns": [
              {
                "name": "VendorID",
                "type": "Nvarchar(max)"
              },
              {
                "name": "tpep_pickup_datetime",
                "type": "DateTime"
              },
              {
                "name": "tpep_dropoff_datetime",
                "type": "DateTime"
              },
              {
                "name": "passenger_count",
                "type": "Float"
              },
              {
                "name": "trip_distance",
                "type": "Float"
              },
              {
                "name": "RatecodeID",
                "type": "Float"
              },
              {
                "name": "store_and_fwd_flag",
                "type": "Nvarchar(max)"
              },
              {
                "name": "PULocationID",
                "type": "Nvarchar(max)"
              },
              {
                "name": "DOLocationID",
                "type": "Nvarchar(max)"
              },
              {
                "name": "payment_type",
                "type": "BigInt"
              },
              {
                "name": "fare_amount",
                "type": "Float"
              },
              {
                "name": "extra",
                "type": "Float"
              },
              {
                "name": "mta_tax",
                "type": "Float"
              },
              {
                "name": "tip_amount",
                "type": "Float"
              },
              {
                "name": "tolls_amount",
                "type": "Float"
              },
              {
                "name": "improvement_surcharge",
                "type": "Float"
              },
              {
                "name": "total_amount",
                "type": "Float"
              },
              {
                "name": "congestion_surcharge",
                "type": "Float"
              },
              {
                "name": "airport_fee",
                "type": "Float"
              }
            ]
          }
        }
      ]
    },
    {
      "name": "DataActivator",
      "type": "Activator",
      "properties": {
        "workspaceId": f"{workspaceId}",
        "itemId": f"{DataActivatorId}",
        "inputSerialization": {
          "type": "Json",
          "properties": {
            "encoding": "UTF8"
          }
        }
      },
      "inputNodes": [
        {
          "name": "ManageFields"
        }
      ],
      "inputSchemas": [
        {
          "name": "ManageFields",
          "schema": {
            "columns": [
              {
                "name": "VendorID",
                "type": "Nvarchar(max)"
              },
              {
                "name": "tpep_pickup_datetime",
                "type": "DateTime"
              },
              {
                "name": "tpep_dropoff_datetime",
                "type": "DateTime"
              },
              {
                "name": "passenger_count",
                "type": "Float"
              },
              {
                "name": "trip_distance",
                "type": "Float"
              },
              {
                "name": "RatecodeID",
                "type": "Float"
              },
              {
                "name": "store_and_fwd_flag",
                "type": "Nvarchar(max)"
              },
              {
                "name": "PULocationID",
                "type": "Nvarchar(max)"
              },
              {
                "name": "DOLocationID",
                "type": "Nvarchar(max)"
              },
              {
                "name": "payment_type",
                "type": "BigInt"
              },
              {
                "name": "fare_amount",
                "type": "Float"
              },
              {
                "name": "extra",
                "type": "Float"
              },
              {
                "name": "mta_tax",
                "type": "Float"
              },
              {
                "name": "tip_amount",
                "type": "Float"
              },
              {
                "name": "tolls_amount",
                "type": "Float"
              },
              {
                "name": "improvement_surcharge",
                "type": "Float"
              },
              {
                "name": "total_amount",
                "type": "Float"
              },
              {
                "name": "congestion_surcharge",
                "type": "Float"
              },
              {
                "name": "airport_fee",
                "type": "Float"
              }
            ]
          }
        }
      ]
    }
  ],
  "streams": [
    {
      "name": "Taxi-Stream",
      "type": "DefaultStream",
      "properties": {},
      "inputNodes": [
        {
          "name": "Taxi"
        }
      ]
    }
  ],
  "operators": [
    {
      "name": "ManageFields",
      "type": "ManageFields",
      "inputNodes": [
        {
          "name": "Taxi-Stream"
        }
      ],
      "properties": {
        "columns": [
          {
            "type": "Rename",
            "properties": {
              "column": {
                "expressionType": "ColumnReference",
                "columnName": "VendorID",
                "columnPathSegments": []
              }
            },
            "alias": "VendorID"
          },
          {
            "type": "Cast",
            "properties": {
              "targetDataType": "DateTime",
              "column": {
                "expressionType": "ColumnReference",
                "columnName": "tpep_pickup_datetime",
                "columnPathSegments": []
              }
            },
            "alias": "tpep_pickup_datetime"
          },
          {
            "type": "Cast",
            "properties": {
              "targetDataType": "DateTime",
              "column": {
                "expressionType": "ColumnReference",
                "columnName": "tpep_dropoff_datetime",
                "columnPathSegments": []
              }
            },
            "alias": "tpep_dropoff_datetime"
          },
          {
            "type": "Cast",
            "properties": {
              "targetDataType": "Float",
              "column": {
                "expressionType": "ColumnReference",
                "columnName": "passenger_count",
                "columnPathSegments": []
              }
            },
            "alias": "passenger_count"
          },
          {
            "type": "Cast",
            "properties": {
              "targetDataType": "Float",
              "column": {
                "expressionType": "ColumnReference",
                "columnName": "trip_distance",
                "columnPathSegments": []
              }
            },
            "alias": "trip_distance"
          },
          {
            "type": "Cast",
            "properties": {
              "targetDataType": "Float",
              "column": {
                "expressionType": "ColumnReference",
                "columnName": "RatecodeID",
                "columnPathSegments": []
              }
            },
            "alias": "RatecodeID"
          },
          {
            "type": "Rename",
            "properties": {
              "column": {
                "expressionType": "ColumnReference",
                "columnName": "store_and_fwd_flag",
                "columnPathSegments": []
              }
            },
            "alias": "store_and_fwd_flag"
          },
          {
            "type": "Rename",
            "properties": {
              "column": {
                "expressionType": "ColumnReference",
                "columnName": "PULocationID",
                "columnPathSegments": []
              }
            },
            "alias": "PULocationID"
          },
          {
            "type": "Rename",
            "properties": {
              "column": {
                "expressionType": "ColumnReference",
                "columnName": "DOLocationID",
                "columnPathSegments": []
              }
            },
            "alias": "DOLocationID"
          },
          {
            "type": "Cast",
            "properties": {
              "targetDataType": "BigInt",
              "column": {
                "expressionType": "ColumnReference",
                "columnName": "payment_type",
                "columnPathSegments": []
              }
            },
            "alias": "payment_type"
          },
          {
            "type": "Cast",
            "properties": {
              "targetDataType": "Float",
              "column": {
                "expressionType": "ColumnReference",
                "columnName": "fare_amount",
                "columnPathSegments": []
              }
            },
            "alias": "fare_amount"
          },
          {
            "type": "Cast",
            "properties": {
              "targetDataType": "Float",
              "column": {
                "expressionType": "ColumnReference",
                "columnName": "extra",
                "columnPathSegments": []
              }
            },
            "alias": "extra"
          },
          {
            "type": "Cast",
            "properties": {
              "targetDataType": "Float",
              "column": {
                "expressionType": "ColumnReference",
                "columnName": "mta_tax",
                "columnPathSegments": []
              }
            },
            "alias": "mta_tax"
          },
          {
            "type": "Cast",
            "properties": {
              "targetDataType": "Float",
              "column": {
                "expressionType": "ColumnReference",
                "columnName": "tip_amount",
                "columnPathSegments": []
              }
            },
            "alias": "tip_amount"
          },
          {
            "type": "Cast",
            "properties": {
              "targetDataType": "Float",
              "column": {
                "expressionType": "ColumnReference",
                "columnName": "tolls_amount",
                "columnPathSegments": []
              }
            },
            "alias": "tolls_amount"
          },
          {
            "type": "Cast",
            "properties": {
              "targetDataType": "Float",
              "column": {
                "expressionType": "ColumnReference",
                "columnName": "improvement_surcharge",
                "columnPathSegments": []
              }
            },
            "alias": "improvement_surcharge"
          },
          {
            "type": "Cast",
            "properties": {
              "targetDataType": "Float",
              "column": {
                "expressionType": "ColumnReference",
                "columnName": "total_amount",
                "columnPathSegments": []
              }
            },
            "alias": "total_amount"
          },
          {
            "type": "Cast",
            "properties": {
              "targetDataType": "Float",
              "column": {
                "expressionType": "ColumnReference",
                "columnName": "congestion_surcharge",
                "columnPathSegments": []
              }
            },
            "alias": "congestion_surcharge"
          },
          {
            "type": "Cast",
            "properties": {
              "targetDataType": "Float",
              "column": {
                "expressionType": "ColumnReference",
                "columnName": "airport_fee",
                "columnPathSegments": []
              }
            },
            "alias": "airport_fee"
          }
        ]
      },
      "inputSchemas": [
        {
          "name": "Taxi-Stream",
          "schema": {
            "columns": [
              {
                "name": "VendorID",
                "type": "Nvarchar(max)"
              },
              {
                "name": "tpep_pickup_datetime",
                "type": "Nvarchar(max)"
              },
              {
                "name": "tpep_dropoff_datetime",
                "type": "Nvarchar(max)"
              },
              {
                "name": "passenger_count",
                "type": "Nvarchar(max)"
              },
              {
                "name": "trip_distance",
                "type": "Nvarchar(max)"
              },
              {
                "name": "RatecodeID",
                "type": "Nvarchar(max)"
              },
              {
                "name": "store_and_fwd_flag",
                "type": "Nvarchar(max)"
              },
              {
                "name": "PULocationID",
                "type": "Nvarchar(max)"
              },
              {
                "name": "DOLocationID",
                "type": "Nvarchar(max)"
              },
              {
                "name": "payment_type",
                "type": "Nvarchar(max)"
              },
              {
                "name": "fare_amount",
                "type": "Nvarchar(max)"
              },
              {
                "name": "extra",
                "type": "Nvarchar(max)"
              },
              {
                "name": "mta_tax",
                "type": "Nvarchar(max)"
              },
              {
                "name": "tip_amount",
                "type": "Nvarchar(max)"
              },
              {
                "name": "tolls_amount",
                "type": "Nvarchar(max)"
              },
              {
                "name": "improvement_surcharge",
                "type": "Nvarchar(max)"
              },
              {
                "name": "total_amount",
                "type": "Nvarchar(max)"
              },
              {
                "name": "congestion_surcharge",
                "type": "Nvarchar(max)"
              },
              {
                "name": "airport_fee",
                "type": "Nvarchar(max)"
              }
            ]
          }
        }
      ]
    }
  ],
  "compatibilityLevel": "1.0"
}

eventStreamProperties = json.dumps(eventStreamProperties)

eventStreamProperties_string = eventStreamProperties.encode('utf-8')
eventStreamProperties_bytes = base64.b64encode(eventStreamProperties_string)
eventStreamPropertiess_string = eventStreamProperties_bytes.decode('utf-8')

StatementMeta(, a40c28b3-b5ab-445c-a790-10036d0850e9, 52, Finished, Available, Finished)

In [51]:
url = f"v1/workspaces/{workspaceId}/items"

payload = {
 "definition": {
  "parts": [
   {
    "path": "eventstream.json",
    "payload": f"{eventStreamPropertiess_string}",
    "payloadType": "InlineBase64"
   },
   {
    "path": ".platform",
    "payload": f"{eventStreamPlatform_string}",
    "payloadType": "InlineBase64"
   }
  ]
 }
}


response=client.post(url,json=payload)

StatementMeta(, a40c28b3-b5ab-445c-a790-10036d0850e9, 53, Finished, Available, Finished)

In [ ]:
print(f"Create request status code {response.status_code}")
print(response.headers['Location'])
async_result_polling_url = response.headers['Location']

while True:
    async_response = client.get(async_result_polling_url)
    async_status = async_response.json().get('status').lower()
    print("Long running operation status " + async_status)
    if async_status != 'running':
        break
   
    time.sleep(3)

print("Long running operation reached terminal state '" + async_status +"'")

if async_status == 'succeeded':
    print("The operation completed successfully.")
    final_result_url= async_response.headers['Location']
    final_result = client.get(final_result_url)
    print(f"Final result: {final_result.json()}")
elif async_status == 'failed':
    print("The operation failed.")
else:
    print("The operation is in an unexpected state:", status)

